In [1]:
from Camera import Camera
import numpy as np
from Frame import Frame

import matplotlib.pyplot as plt
from GenerateGsInput import GenerateGsInput
import Plotters
import plotly.graph_objects as go
import scipy.io
import pandas as pd
import Utils

%matplotlib qt

# load hull
path = 'I:/My Drive/Research/gs_data/mov19_2022_03_03/'
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}


In [2]:
# initilize objects
frames = [550]#list(range(500,700,13))

image_name,points_in_idx = Utils.define_frames(frames,points_3d)
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {im_name:Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_name)}

In [3]:
# map 3d voxels to 2d pixels
[frames[im_name].map_3d_2d(croped_image = True) for im_name in frames.keys()]
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)

ggs = GenerateGsInput(path,frames)
ggs.save_croped_images(croped_image = False)
ggs.generate_camera_text(croped_image = False)
ggs.generate_image_text()
ggs.generate_points3d_text(voxel_dict,colors_dict)



In [13]:
im_name = list(frames.keys())[0]
fig = go.Figure()
Plotters.scatter3d(fig,frames[im_name].points_in_ew_frame,'red',3)
fig.show()


In [14]:
homo_voxels_with_idx = frames[image].add_homo_coords(frames[image].voxels_with_idx[:,0:3])
homo_voxels_with_idx.shape

NameError: name 'image' is not defined

In [15]:
# plots
# camera ptcloud
color_pts = ['red','green','blue','black']
fig = go.Figure()
im_name = list(frames.keys())[0]
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    Plotters.scatter3d(fig,frames[image].voxels_with_idx,color_pts[cam],3)
fig.show()



# projection
fig,axs = plt.subplots(2,2)
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    homo_voxels_with_idx = frames[image].add_homo_coords(frames[image].voxels_with_idx[:,0:3])
    proj = frames[image].project_on_image(homo_voxels_with_idx,croped_camera_matrix = False)
    axs[cam // 2,cam % 2].imshow(frames[image].image)
    axs[cam // 2,cam % 2].scatter(proj[:,0],proj[:,1],s = 3,c = 'red')


In [6]:
from colmap_loader import read_extrinsics_text, read_intrinsics_text, qvec2rotmat, \
    read_extrinsics_binary, read_intrinsics_binary, read_points3D_binary, read_points3D_text


cameras_extrinsic_file = f'{ggs.sparse_dir}/images.txt'
cameras_intrinsic_file = f'{ggs.sparse_dir}/cameras.txt'
txt_path =f'{ggs.sparse_dir}/points3D.txt'

xyz, rgb, _ = read_points3D_text(txt_path)

cam_extrinsics = read_extrinsics_text(cameras_extrinsic_file)
cam_intrinsics = read_intrinsics_text(cameras_intrinsic_file)

from PIL import Image

image = Image.open(f'{ggs.image_dir}/{im_name}')
image.mode


'RGB'

In [7]:
from colmap_loader import read_extrinsics_text, read_intrinsics_text, qvec2rotmat, \
    read_extrinsics_binary, read_intrinsics_binary, read_points3D_binary, read_points3D_text


cameras_extrinsic_file = f'{ggs.sparse_dir}/images.txt'
cameras_intrinsic_file = f'{ggs.sparse_dir}/cameras.txt'
txt_path =f'{ggs.sparse_dir}/points3D.txt'

xyz, rgb, _ = read_points3D_text(txt_path)

cam_extrinsics = read_extrinsics_text(cameras_extrinsic_file)
cam_intrinsics = read_intrinsics_text(cameras_intrinsic_file)

from PIL import Image

image = Image.open(f'{ggs.image_dir}/{im_name}')
image.mode


'RGB'

In [9]:
args = np.load('output_arrays.npz')  # Save all arrays in a single file


In [130]:
frames[image_name].K[0,2]

644.1280044541276

In [30]:
import math

def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])



def getWorld2View2(R, t, translate=np.array([.0, .0, .0]), scale=1.0):
    Rt = np.zeros((4, 4))
    Rt[:3, :3] = R.transpose()
    Rt[:3, 3] = t
    Rt[3, 3] = 1.0

    C2W = np.linalg.inv(Rt)
    cam_center = C2W[:3, 3]
    cam_center = (cam_center + translate) * scale
    C2W[:3, 3] = cam_center
    Rt = np.linalg.inv(C2W)
    return np.float32(Rt)



def focal2fov(focal, pixels):
    return 2*math.atan(pixels/(2*focal))

def getProjectionMatrix1(znear, zfar, fovX, fovY,cx,cy,im_size):
    tanHalfFovY = math.tan((fovY / 2))
    tanHalfFovX = math.tan((fovX / 2))

    top = tanHalfFovY * znear
    bottom = -top
    right = tanHalfFovX * znear
    left = -right

    P = np.zeros((4, 4))

    z_sign = 1.0

    P[0, 0] = 2.0 * znear / (right - left)
    P[1, 1] = 2.0 * znear / (top - bottom)
    P[0, 2] = (right + left) / (right - left)
    P[1, 2] = (top + bottom) / (top - bottom)
    P[3, 2] = z_sign
    P[2, 2] = z_sign * zfar / (zfar - znear)
    P[2, 3] = -(zfar * znear) / (zfar - znear)
    P[0, 2] = (right + left) / (right - left) + (2 * cx / im_size[0]) - 1
    P[1, 2] = (top + bottom) / (top - bottom) + (2 * cy / im_size[1]) - 1
    return P

def getProjectionMatrixShift(znear, zfar, focal_x, focal_y, cx, cy, width, height, fovX, fovY):
    tanHalfFovY = math.tan((fovY / 2))
    tanHalfFovX = math.tan((fovX / 2))

    # the origin at center of image plane
    top = tanHalfFovY * znear
    bottom = -top
    right = tanHalfFovX * znear
    left = -right

    # shift the frame window due to the non-zero principle point offsets
    offset_x = cx - (width/2)
    offset_x = (offset_x/focal_x)*znear
    offset_y = cy - (height/2)
    offset_y = (offset_y/focal_y)*znear

    top = top + offset_y
    left = left + offset_x
    right = right + offset_x
    bottom = bottom + offset_y

    P = np.zeros((4, 4))

    z_sign = 1.0

    P[0, 0] = 2.0 * znear / (right - left)
    P[1, 1] = 2.0 * znear / (top - bottom)
    P[0, 2] = (right + left) / (right - left)
    P[1, 2] = (top + bottom) / (top - bottom)
    P[3, 2] = z_sign
    P[2, 2] = z_sign * zfar / (zfar - znear)
    P[2, 3] = -(zfar * znear) / (zfar - znear)
    return P

def adjust_focal_lengths(fx, fy, cx, cy, image_width, image_height, znear):
    # Shifts from the image center
    shift_x = cx - image_width / 2
    shift_y = cy - image_height / 2
    
    # Adjust the focal lengths based on the shifts
    adjusted_fx = fx - (shift_x / znear)
    adjusted_fy = fy - (shift_y / znear)
    
    return adjusted_fx, adjusted_fy


def adjust_translation(t, cx, cy, image_width, image_height):
    # Adjust the translation based on the shift from center
    t_adjusted = np.copy(t)
    t_adjusted[0] += (cx - image_width / 2) * 1  # Adjust for x shift
    t_adjusted[1] += (cy - image_height / 2) * 1 # Adjust for y shift
    
    return t_adjusted


def getProjectionMatrix(znear, zfar, fovX, fovY):
    tanHalfFovY = math.tan((fovY / 2))
    tanHalfFovX = math.tan((fovX / 2))

    top = tanHalfFovY * znear
    bottom = -top
    right = tanHalfFovX * znear
    left = -right

    P = np.zeros((4, 4))

    z_sign = 1.0

    P[0, 0] = 2.0 * znear / (right - left)
    P[1, 1] = 2.0 * znear / (top - bottom)
    P[0, 2] = (right + left) / (right - left)
    P[1, 2] = (top + bottom) / (top - bottom)
    P[3, 2] = z_sign
    P[2, 2] = z_sign * zfar / (zfar - znear)
    P[2, 3] = -(zfar * znear) / (zfar - znear)
    P[0, 2] = (right + left) / (right - left) 
    P[1, 2] = (top + bottom) / (top - bottom) 
    return P

image_name = 'P550CAM2.jpg'



image_width = 1280
image_height = 800
znear = 0.001
zfar = 100

focalx = frames[image_name].K[0,0]
focaly = frames[image_name].K[1,1]
cx = frames[image_name].K[0, 2]  # Updated cx for cropped image
cy = frames[image_name].K[1, 2]  # Updated cy for cropped image


R = qvec2rotmat(frames[image_name].qvec).T
# R = frames[image_name].R
T = frames[image_name].t.T

im_size = [1280,800]

# focalx,focaly = adjust_focal_lengths(focalx, focaly, cx, cy, image_width, image_height, znear)
fovX = focal2fov(focalx, image_width)
fovY = focal2fov(focaly, image_height)

world_view_transform = getWorld2View2(R, T)
# projection_matrix = getProjectionMatrix(znear, zfar, fovX, fovY)
projection_matrix = getProjectionMatrix1(znear, zfar, fovX, fovY,cx,cy,im_size)

projection_matrix = getProjectionMatrixShift(znear, zfar, focalx, focaly, cx, cy, image_width, image_height, fovX, fovY)
full_proj_transform = np.matmul(projection_matrix,world_view_transform)  # Shape: (1, N, K)
# full_proj_transform = np.dot(projection_matrix,world_view_transform)

xyz_homo  = np.column_stack((frames[image_name].voxels_with_idx[:,0:3],np.ones((frames[image_name].voxels_with_idx.shape[0],1))))

p_proj = np.matmul(full_proj_transform,xyz_homo.T).T
p_proj = p_proj[:,0:3]/p_proj[:,3:]

def proj_screen(pixel,s):
    return ((pixel + 1)*s-1)*0.5
pixel_x =proj_screen(p_proj[:,0],image_width)
pixel_y =proj_screen(p_proj[:,1],image_height)
plt.figure()
plt.imshow(frames[image_name].image)

plt.scatter(pixel_x,pixel_y)



: 

In [22]:
T

array([[-0.00187092, -0.00162703,  0.19135575]])

In [15]:
wak = [[-7.0028e+00, -6.0958e-03, -1.9097e-01, -1.3016e-02],
        [-9.6542e-03,  1.1228e+01, -6.1573e-04,  3.1913e-03],
        [ 2.6349e-02, -9.9836e-03, -9.9970e-01,  1.8125e-01],
        [ 2.6347e-02, -9.9826e-03, -9.9960e-01,  1.9123e-01]]

In [13]:
full_proj_transform = np.matmul(projection_matrix,world_view_transform)  # Shape: (1, N, K)
full_proj_transform
# full_proj_transform = np.matmul(projection_matrix,world_view_transform)  # Shape: (1, N, K)
# projection_matrix.T

array([[-7.00280764e+00, -6.09581507e-03, -1.90966743e-01,
        -1.30163719e-02],
       [-9.65435123e-03,  1.12281467e+01, -6.09909441e-04,
         3.19020648e-03],
       [ 2.63470923e-02, -9.98269877e-03, -9.99613089e-01,
         1.90236506e-01],
       [ 2.63468288e-02, -9.98259895e-03, -9.99603093e-01,
         1.91234604e-01]])

In [23]:
world_view_transform

array([[-0.47209173,  0.5003789 , -0.72577566, -0.00187092],
       [ 0.51598847, -0.51068443, -0.68771887, -0.00162703],
       [-0.7147624 , -0.6991583 , -0.01709984,  0.19135575],
       [ 0.        ,  0.        ,  0.        ,  1.        ]],
      dtype=float32)

In [44]:
frames['P550CAM1.jpg'].points_in_ew_frame

array([[-1.2000e-03, -1.1400e-02,  7.5000e-04,  1.0000e+00],
       [-1.1500e-03, -1.1550e-02,  5.0000e-04,  2.0000e+00],
       [-1.1500e-03, -1.1550e-02,  5.5000e-04,  3.0000e+00],
       ...,
       [ 1.5000e-04, -1.1250e-02,  4.0000e-04,  2.5250e+04],
       [ 1.0000e-04, -1.1200e-02,  4.0000e-04,  2.5251e+04],
       [ 5.0000e-05, -1.1150e-02,  4.0000e-04,  2.5252e+04]])

In [9]:
full_proj_transform

array([[ 3.31713426, -3.62557293,  5.02224916,  0.01314593],
       [-5.61027528,  5.7258234 ,  7.83900321,  0.0182424 ],
       [-0.7258483 , -0.68778765, -0.01710146,  0.18137389],
       [-0.72577572, -0.68771887, -0.01709975,  0.19135575]])

array([[-0.47209181,  0.51598852, -0.71476232],
       [ 0.5003788 , -0.51068448, -0.69915837],
       [-0.72577571, -0.68771885, -0.01709975]])

In [97]:

cam_pts = np.matmul(world_view_transform,xyz_homo.T).T
p_proj = np.matmul(projection_matrix,cam_pts.T).T
# p_proj = p_proj[:,0:3]/p_proj[:,3:]
p_proj
pw = 1/p_proj[:,3:]
p_proj*pw

array([[-1.88611907, -0.30470322,  0.95095231,  1.        ],
       [-1.88303771, -0.32173837,  0.95096031,  1.        ],
       [-1.86515488, -0.32563956,  0.95098567,  1.        ],
       ...,
       [-1.96160916, -1.08382571,  0.95046296,  1.        ],
       [-1.95184981, -1.09382338,  0.95046258,  1.        ],
       [-1.93023197, -1.07320906,  0.95047176,  1.        ]])

In [87]:
p_proj = np.matmul(full_proj_transform,xyz_homo.T).T
p_proj = p_proj[:,0:2]/p_proj[:,2:]

plt.plot(p_proj)

In [67]:
world_view_transform

array([[-5.1136428e-01, -4.9903232e-01,  6.9962370e-01,  2.3405244e-06],
       [-4.6974885e-01, -5.1940536e-01, -7.1383059e-01, -3.1033736e-03],
       [ 7.1961284e-01, -6.9367486e-01,  3.1185472e-02,  1.9604126e-01],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]],
      dtype=float32)

In [21]:
ptcld = np.load('D:/Documents/points.npy')  # Save all arrays in a single file
fig = go.Figure()
Plotters.scatter3d(fig,ptcld,'red',3)
Plotters.scatter3d(fig,frames['P550CAM2.jpg'].voxels_with_idx,'blue',3)
fig.show()

In [9]:
color_pts = ['red','green','blue','black']
fig = go.Figure()
im_name = list(frames.keys())[0]
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    Plotters.scatter3d(fig,frames[image].voxels_with_idx,color_pts[cam],3)
    Plotters.scatter3d(fig,frames[image].points_in_ew_frame_homo,color_pts[cam],3)

fig.show()


In [36]:
np.asarray(point_cloud.colors)

array([], shape=(0, 3), dtype=float64)

In [35]:
point_cloud

PointCloud with 1070786 points.

In [13]:
import open3d as o3d

# Load the PLY file
ply_file_path = "D:/Documents/point_cloud.ply"
point_cloud = o3d.io.read_point_cloud(ply_file_path)

# Visualize the point cloud
# o3d.visualization.draw_geometries([point_cloud])

ModuleNotFoundError: No module named 'open3d'

In [3]:
homo_c = np.matmul(proj,xyz_homo.T).T


p_proj = np.matmul(full_proj_transform.T,homo_c.T).T

p_proj = p_proj[:,0:3]/p_proj[:,3:]
p_proj
# m_x= p_proj[:,0]
# m_y = p_proj[:,1]
# m_z = p_proj[:,2]
# m_w = 1/p_proj[:,3]
def proj_screen(pixel,s):
    return ((pixel + 1)*s-1)*0.5
pixel_x =proj_screen(p_proj[:,0],160)
pixel_y =proj_screen(p_proj[:,1],160)
plt.imshow(frames[image_name].croped_image)

plt.scatter(pixel_x,pixel_y)

NameError: name 'np' is not defined

In [4]:
wakk =  np.column_stack((frames[image].voxels_with_idx[:,0:3]/frames[image].voxels_with_idx[:,3:4],np.ones((frames[image].voxels_with_idx.shape[0],1))))
wakk

NameError: name 'np' is not defined

In [5]:


mul1 = (proj[0,0] * m_x + proj[1,0] * m_y + proj[2,0] * m_z + proj[3,0])*m_w
mul2 = (proj[0,1] * m_x + proj[1,1] * m_y + proj[2,1] * m_z + proj[3,1])*m_w

plt.scatter(mul1,mul2)

NameError: name 'proj' is not defined

In [6]:
	dL_dmean.x = (proj[0] * m_w - proj[3] * mul1) * dL_dmean2D[idx].x + (proj[1] * m_w - proj[3] * mul2) * dL_dmean2D[idx].y;
	dL_dmean.y = (proj[4] * m_w - proj[7] * mul1) * dL_dmean2D[idx].x + (proj[5] * m_w - proj[7] * mul2) * dL_dmean2D[idx].y;

NameError: name 'proj' is not defined

In [81]:
frames['P550CAM2.jpg'].world_to_cam.T.shape

(4, 3)

In [7]:
frames['P550CAM3.jpg'].t

NameError: name 'frames' is not defined

In [8]:
wakk

NameError: name 'wakk' is not defined

array([[-0.47209181,  0.5003788 , -0.72577571,  0.        ],
       [ 0.51598852, -0.51068448, -0.68771885,  0.        ],
       [-0.71476232, -0.69915837, -0.01709975,  0.        ],
       [-0.00187092, -0.00162703,  0.19135575,  1.        ]])

In [9]:
args['arr_8']

NameError: name 'args' is not defined

In [43]:
args['arr_9']

array([[ 2.8788988e+01,  2.6489201e+01,  7.1968478e-01,  7.1961278e-01],
       [ 2.8094706e+01,  2.9289333e+01, -6.9374442e-01, -6.9367504e-01],
       [-3.9387680e+01,  4.0252998e+01,  3.1188568e-02,  3.1185448e-02],
       [-1.3176772e-04,  1.7499962e-01,  1.8605986e-01,  1.9604126e-01]],
      dtype=float32)

In [22]:
args['arr_1']

fig = go.Figure()

Plotters.scatter3d(fig,frames['P550CAM1.jpg'].voxels_with_idx,'blue',3)
Plotters.scatter3d(fig,args['arr_1'],'red',3)

fig.show()

In [23]:
image_rgb = image.convert('RGB')
image_rgb.save('wakk.jpg')


In [24]:
image.size

pt = 'D:\Documents\south-building\images\P1180141.jpg'
image = Image.open('wakk.jpg')
image.mode

'RGB'